<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Employee_Scheduling_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Employee Scheduling Optimization

## Objective and Prerequisites

This Employee Scheduling Optimization problem shows you how to determine the optimal number of employees on the payroll in order to:

* Satisfy demand for each day's staffing requirement,
* Minimize the payroll cost of all employees, and
* Prescribe the actual staffing levels achieved for each day.

This modeling example is at the introductory level, where we assume that you know Python and that you have some knowledge of how to build mathematical optimization models.

---
## Problem Description

A small business requires different numbers of full-time employees on different days of the week. The company implements a flexible working policy, which states that each full-time employee can work any five days and take two days off in a week. For example, an employee can work on Monday to Thursday and Saturday while taking Friday and Sunday off. The company wants to meet its daily requirements using only full-time employees. The employees are paid \$200 pay day if they work on regular weekdays while receive an additional \$100 per day if they work over the weekend. The company's objective is to minimize its payroll expenses.

![picture](https://drive.google.com/uc?id=1zni6oEycubocdW7fLglYB6pGwu7hHkdw)

The following table lists the staffing requirements on each day of the week.

| Day of the Week | Required Staffing Level |
| --- | --- |
| Monday    | 177 |
| Tuesday   | 134 |
| Wednesday | 158 |
| Thursday  | 191 |
| Friday    | 149 |
| Saturday  | 165 |
| Sunday    | 116 |

In this example, the goal is to identify how many workers to hire and schedule them so that the total payroll cost is minimized. This example shows how a linear programming model model can help the business to:

* How to best utilize their employees, 
* How many workers to hire, and
* How they should be scheduled so that the total payroll cost is miniminzed. This Jupyter Notebook is based on the MSBA SCM518 class contents.

## Model Formulation

### Indices

$i \in \{1..21\}$: Index of possible working schedules (e.g., which days an employee works and which days are off).

$j \in \{1..7\}$: Index of work days.

### Parameters

$d_{j}$: Required staffing level on day $j$.

$p_R$: Daily pay for weekdays.

$p_O$: Daily pay for weekends.

$p_{j}$: Daily pay for day $j$.

$a_{ij} \in \{0,1\}$: An availability table that captures whether employees working under schedule $i$ is available for day $j$.

### Decision Variables

$x_{i}$: How many employees to hire for work schedule $i$.


### Objective Function

- **Cost**. We want to minimize the total payroll cost.


\begin{equation}
\text{Min}_{x_i} \quad \sum_{j \in \{1...7\}} p_j \left(\sum_{i \in \{1...21\}} a_{ij}*x_i\right)
\tag{0}
\end{equation}

### Constraints

- **Staffing Requirement**. Satisfy staffing requirement for each day.

\begin{equation}
\sum_{i \in \{1...21\}} a_{i,j}*x_{i} \geq d_j \quad \forall j \in \{1...7\}
\tag{1}
\end{equation}

\begin{equation}
x_i \geq 0 \quad \forall i \in \{1...21\}
\tag{2}
\end{equation}

---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

---

### Helper Functions

We first create a function that helps us to generate an employee availability table based on different work schedules

In [ ]:
# compute number of cominations 
def nCr(n, r): 
    return int(factorial(n) / (factorial(r) * factorial(n - r)))
  
# compute availability table
# w is the length of the shift, d is number of days off 
def schedule(w,d):  

    # number of shifts 
    v = nCr(w,d)
    s = [[1 for x in range(w)] for x in range(v)] 

    i=0
    for j in range(v):
        for k in range(j+1,w):
            s[i][j] = 0
            s[i][k] = 0   
            i=i+1
    return s      


Generate the availability table and take a look at the table

In [ ]:
# generate the schedule availability table
a = schedule(7,2)

# print the table to visualize
print(np.matrix(a))

Set up the model and solve

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('employee scheduling')

shift = [*range(0,21)]
day = [*range(0,7)]

p = [200,200,200,200,200,300,300]

d=[177,134,158,191,149,165,116]

# Cartesian product of shifts and days
sd = []
    
for i in shift:
    for j in day:
        tp = i,j
        sd.append(tp)

#print(np.matrix(sd))

# Build decision variables: how many employees to work on a shift
x = m.addVars(shift, vtype=GRB.INTEGER, name='Assign')
    
# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(p[j]*a[i][j]*x[i] for i,j in sd), GRB.MINIMIZE)
    
# Satisfying staffing requirements
staffingConstrs = m.addConstrs((gp.quicksum(x[i]*a[i][j]  for i in shift) >= d[j] for j in day), 
                                      name='staffingConstrs')
    
# Run optimization engine
m.optimize()

Take a look at the results on number of employees on the payroll

In [ ]:
#####################################################
#         Number of employees on each shift
#####################################################
    
print(f"\n\n___Optimal employees on each shift________")
total_employee = 0
for t in shift:
    print("The number of employee on shift %2d is %3d" % (t, x[t].x))
    #print(f"The number of employee on shift {t} is {x[t].x}") 
    total_employee += x[t].x
            
print("The total number of emploee is %3d " % (total_employee))  


Take a look at the staffing coverage for each day of the week.

In [ ]:
#####################################################
#     Actual coverage on each day
#####################################################
    
print(f"\n\n____Actual coverage on each day_____")
for t in day:
    actual_employee = 0
    for s in shift:
      actual_employee += (x[s].x)*a[s][t]
    print("The number of employee on shift %1d is %3d where demand is %3d" % (t, actual_employee, d[t]) ) 

###A variant of the model where employees must take two consequtive days off.

The mathematical model remains identical, where the only change is the availability table: instead of C(7,2)=21 rows, we now only have C(7,1)=7 rows. See below for details. Thus, there is no need to setup a separate mathematical model.

First compute the availability table.

In [ ]:
# assuming employees must take two consequtive days off
# compute availability table
# w is the length of the shift, d is number of days off 
def schedule(w,d):  

    s = [[1 for x in range(w)] for x in range(w)] 
    i=0
    for j in range(w):
        s[i][j] = 0
        s[i][(j+1)%w] = 0   
        i=i+1
    return s      

Generate the availability table and take a look.

In [ ]:
# generate the schedule availability table
a = schedule(7,2)

# print the table to visualize
print(np.matrix(a))

Setup the model

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('employee scheduling')

shift = [*range(0,7)]
day = [*range(0,7)]

p = [200,200,200,200,200,300,300]

d=[177,134,158,191,149,165,116]

# Cartesian product of shifts and days
sd = []
    
for i in shift:
    for j in day:
        tp = i,j
        sd.append(tp)

#print(np.matrix(sd))

# Build decision variables: how many employees to work on a shift
x = m.addVars(shift, vtype=GRB.INTEGER, name='Assign')
    
# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(p[j]*a[i][j]*x[i] for i,j in sd), GRB.MINIMIZE)
    
# Satisfying staffing requirements
staffingConstrs = m.addConstrs((gp.quicksum(x[i]*a[i][j]  for i in shift) >= d[j] for j in day), 
                                      name='staffingConstrs')
    
# Run optimization engine
m.optimize()

Take a look at the number of employees hired.

In [ ]:
#####################################################
#         Number of employees on each shift
#####################################################
    
print(f"\n\n___Optimal employees on each shift________")
total_employee = 0
for t in shift:
    print("The number of employee on shift %2d is %3d" % (t, x[t].x))
    #print(f"The number of employee on shift {t} is {x[t].x}") 
    total_employee += x[t].x
            
print("The total number of emploee is %3d " % (total_employee))  


Take a look at the actual coverage.

In [ ]:
#####################################################
#     Actual coverage on each day
#####################################################
    
print(f"\n\n____Actual coverage on each day_____")
for t in day:
    actual_employee = 0
    for s in shift:
      actual_employee += (x[s].x)*a[s][t]
    print("The number of employee on shift %1d is %3d where demand is %3d" % (t, actual_employee, d[t]) ) 

As we can see, when the employee schedules are less flexible (two consequtive days off), the company needs to hire more people to satisfy daily demand. This is consistent with the theoretical predication: when more constraints are imposed, the objective can only get worse.

---
##  Conclusion

In this example, we addressed the employee scheduling problem. We determined the optimal number of employee to: 
* Satisfy demand for each day, 
* Minimize the total number of employees, and 
* Find number of employees to place in each shift.

We also consider variations of the model where employees must take two conseutive days off.

Our employee scheduling model can be used by many organizations to help make informed decisions about which shifts and how many employees to have in order to satisfy daily demand while minimizing the number of employees on the payroll.


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.

[2] Gurobi python reference. https://www.gurobi.com/documentation/

[3] This notebook is developed by Yimin Wang. If you have any suggestions or comments, please contact yimin_wang@asu.edu